TO-DOs:

INCIDENT FILE
- COMPLETED: Parse-out the time from the date/time columns  (Converted to datetime)
- COMPLETED: Calculate the arrival times
- COMPLETED: Create FINAL dataframe for our assignments

STATION FILE
- COMPLETED: Only contain CLV stations


ASSIGNMENTS:
- COMPLETE: Google map the CLV station on a map?  (Margot)
- OPTIONAL: Add clicable names to the pins?  (Margot)
- COMPLETED: Pie chart for incident type (Saeger)
- COMPLETED: Bar chart by incident count for each station (Saeger)
- COMPLETED: Bar chart by response time for each station (Saeger)
- COMPLETED: Hypothesis Testing (Anji)
        - Boxplots?
        - Determine Mean vs Median based on number of outliers?
- COMPLETED: Trends - number of incidents by month compared to previous years (George)
- COMPLETED: Trends - Avg response time by month compared to previous years (George)
- ?: Pick the highest performing station (lowest avg response times) vs least performing station (highest avg response times)  and look into why  (Further charting for each?)
- TBD: Quality control & code clean-up


In [ ]:
# DEPENDENCIES
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
import json

# Declare variables for each CSV file
incident_file_df1114 = "data/Fire_Department_Incident_Count - 2011-2014.csv"
incident_file_df1516 = "data/Fire_Department_Incident_Count - 2015-2016.csv"
incident_file_df1718 = "data/Fire_Department_Incident_Count - 2017-2018.csv"
fire_station_data =  "data/Clark_County_Fire_Stations.csv"

#import gmaps
import gmaps

# Import API key
from configure import g_key

PROCESS INCIDENT DATA HERE

In [ ]:
# Step 1: bring-in 2011-2014 data into a df

incident_file_df1718 = pd.read_csv(incident_file_df1718)
print(f"# of rows in 2017-2018 DF: {len(incident_file_df1718)}")

incident_file_df1516 = pd.read_csv(incident_file_df1516)
print(f"# of rows in 2014-2015 DF: {len(incident_file_df1516)}")

incident_file_df1114 = pd.read_csv(incident_file_df1114)
print(f"# of rows in 2011-2014 DF: {len(incident_file_df1114)}")

In [ ]:
# Concatonate previous DFs into a single dataframe

incident_file_df = pd.concat([incident_file_df1114, incident_file_df1516, incident_file_df1718])
incident_file_df = incident_file_df.dropna(how='any')
print(f"# of rows in complete file: {len(incident_file_df)}")

In [ ]:
# New 'lean' DF with only the required columns and ignore the others

incident_file_lean_df = incident_file_df[["Station","Response_Date","Event_Type","First_Unit_Assigned","First_Unit_Arrived","Location","Location_1"]]
incident_file_lean_df.head()

In [ ]:
# Create new df for rows that DO NOT contain 'none' in the First_Unit_Arrived column

print(f"Number of rows BEFORE clean-up: {len(incident_file_lean_df)}")
incident_file_clean_df = incident_file_lean_df[incident_file_lean_df["First_Unit_Arrived"] != "None"]
print(f"Number of rows AFTER clean-up: {len(incident_file_clean_df)}")
incident_file_clean_df = incident_file_clean_df.drop_duplicates()
print(f"Number of rows AFTER drop_duplicates: {len(incident_file_clean_df)}")
incident_file_clean_df.head()

In [ ]:
# Convert Assigned & Arrived times to DateTime, remove timezones
incident_file_clean_df["First_Unit_Assigned"] = pd.to_datetime(incident_file_clean_df["First_Unit_Assigned"])
incident_file_clean_df["First_Unit_Arrived"] = pd.to_datetime(incident_file_clean_df["First_Unit_Arrived"])
incident_file_clean_df["First_Unit_Assigned"] = incident_file_clean_df["First_Unit_Assigned"].dt.tz_localize(None)
incident_file_clean_df.head()

In [ ]:
# Calculate Response Time column
incident_file_clean_df["Response Time"] = (incident_file_clean_df["First_Unit_Arrived"] - 
                                           incident_file_clean_df["First_Unit_Assigned"]).astype('timedelta64[m]')
incident_file_clean_df.head()

In [ ]:
#Create FINAL, cleaned dataframe, ready for analysis, charts, maps, and hypothesis testing!!!
print(f"Rows BEFORE final clean-up: {len(incident_file_clean_df)}")
incident_file_final_df = incident_file_clean_df[incident_file_clean_df["First_Unit_Arrived"] > 
                                                incident_file_clean_df["First_Unit_Assigned"]]
print(f"Rows AFTER final clean-up: {len(incident_file_final_df)}")
incident_file_final_df.head()

PROCESS FIRE STATION DATA (MARGOT'S SECTION)

In [ ]:
# Bring-in fire station data
station_df = pd.read_csv(fire_station_data)
print(f"# of rows in the Station data file: {len(station_df)}")
#station_df.head()

In [ ]:
distinct_stations = incident_file_final_df["Station"].unique()
clv_stations_df = station_df[station_df["CITY_CODE"] == "CLV"]
clv_stations_df.head()

INCIDENT TYPES & COUNTS (SAEGER)

In [ ]:
incident_file_final_df["First_Unit_Assigned"]

In [ ]:
incident_file_final_df.head()

In [ ]:
Stations = incident_file_final_df["Station"].value_counts()

In [ ]:
Stations.head(20)

In [ ]:
plot_pandas= Stations.plot(kind = "bar", color = 'red')
plt.xlabel('Station')
plt.ylabel('Number of Calls')
plt.title("Amount of Total Calls by Station")
plt.show()

In [ ]:
Medical_Incidents = incident_file_final_df.loc[incident_file_final_df["Event_Type"] == "Medical"]
Medical_Calls = Medical_Incidents["Station"].value_counts()

Medical_Calls.head()

In [ ]:
Fire_Incidents = incident_file_final_df.loc[incident_file_final_df["Event_Type"] == "Fire"]
Fire_Calls = Fire_Incidents["Station"].value_counts()

Fire_Calls.head()

In [ ]:
Public_Need_Incidents = incident_file_final_df.loc[incident_file_final_df["Event_Type"] == "Public Need"]
Public_Need_Calls = Public_Need_Incidents["Station"].value_counts()

Public_Need_Calls.head()

In [ ]:
Other_Incidents = incident_file_final_df.loc[incident_file_final_df["Event_Type"] == "Other"]
Other_Calls = Other_Incidents["Station"].value_counts()

Other_Calls.head()

In [ ]:
plot_pandas= Medical_Calls.plot(kind = "bar", color = 'blue')
plt.xlabel('Station')
plt.ylabel('Number of Calls')
plt.title("Amount of Medical Calls by Station")
plt.show()

In [ ]:
plot_pandas= Fire_Calls.plot(kind = "bar", color = 'red')
plt.xlabel('Station')
plt.ylabel('Number of Calls')
plt.title("Amount of Fire Calls by Station")
plt.show()

In [ ]:
plot_pandas= Public_Need_Calls.plot(kind = "bar", color = 'green')
plt.xlabel('Station')
plt.ylabel('Number of Calls')
plt.title("Amount of Public Need Calls by Station")
plt.show()

In [ ]:
plot_pandas= Other_Calls.plot(kind = "bar", color = 'orange')
plt.xlabel('Station')
plt.ylabel('Number of Calls')
plt.title("Amount of Other Calls by Station")
plt.show()

In [ ]:
Event_Type = incident_file_final_df["Event_Type"].unique()

In [ ]:
incident_file_final_df.head()

In [ ]:
#Analyzing incidents by Event Type
Calls = incident_file_final_df["Event_Type"].value_counts()

In [ ]:
Calls

In [ ]:
Calls.plot(kind='pie', startangle = 360, autopct ='%1.1f%%', figsize=(7, 7))
plt.title("Event Types")
plt.show()

In [ ]:
Calls.plot.pie(startangle=45,autopct='%1.1f%%', figsize=(7, 7))

In [ ]:
Medicalcheck = incident_file_final_df.loc[incident_file_final_df["Event_Type"] == "Medical"]

In [ ]:
Medicalcheck.head()

In [ ]:
Medical_Incidents = incident_file_final_df.loc[incident_file_final_df["Event_Type"] == "Medical"]
Medical_Calls = Medical_Incidents["Station"].value_counts()

Medical_Calls.head()

In [ ]:
response_df = incident_file_final_df[['First_Unit_Assigned','Response Time','Station']]

In [ ]:
response_df['Year'] = pd.DatetimeIndex(response_df['First_Unit_Assigned']).year

In [ ]:
response_df

In [ ]:
avg_pivoted_df = response_df.pivot_table(index='Station', values ='Response Time',columns='Year',aggfunc='mean')

In [ ]:
avg_pivoted_df

In [ ]:
plot_pandas= avg_pivoted_df.plot(kind = "bar", color = 'purple')
plt.xlabel('Station')
plt.ylabel('Average Response Time')
plt.title("Average Resopnse Time by Station")
plt.show()

In [ ]:
# avg = avg_pivoted_df.plot(kind='line', xlabel='Station', ylabel='Average Response Time (min)', grid=True,
#                           title='Year Over Year Avg Response Time', figsize=(16,12),linewidth=4)
# avg.legend(bbox_to_anchor=(1,1.02), loc='upper left')


RESPONSE TIMES ANALYSIS (GEORGE)

In [ ]:
# New DF with only necessary columns, including Location to help look for duplicate rows
response_df = incident_file_final_df[['First_Unit_Assigned','Response Time']]
#response_df = incident_file_final_df[['Station','First_Unit_Assigned','Response Time','Location']]

# Create Year & Month columns based on First_Unit_Assigned column
response_df['Year'] = pd.DatetimeIndex(response_df['First_Unit_Assigned']).year
response_df['Month'] = pd.DatetimeIndex(response_df['First_Unit_Assigned']).month
#response_df.set_index(['Year','Month'])

response_df.head()

In [ ]:
# Pivot the response_df for AVG RESPONSE TIME and prep for year-over-year line chart
avg_pivoted_df = response_df.pivot_table(index='Month', values ='Response Time',columns='Year',aggfunc='mean')
avg_pivoted_df

In [ ]:
avg = avg_pivoted_df.plot(kind='line', xlabel='Month', ylabel='Average Response Time (min)', grid=True,
                          title='Year Over Year Avg Response Time', figsize=(10,8),linewidth=4)
avg.legend(bbox_to_anchor=(1,1.02), loc='upper left')

plt.show()

In [ ]:
# Pivot the response_df for INCIDENT COUNTS and prep for year-over-year line chart
count_pivoted_df = response_df.pivot_table(index='Month', values ='Response Time',columns='Year',aggfunc='count')
count_pivoted_df

In [ ]:
count = count_pivoted_df.plot(kind='line', xlabel='Month', ylabel='Number of Incidents', grid=True,
                          title='Year Over Year Number of Incidents', figsize=(10,8),linewidth=4)
count.legend(bbox_to_anchor=(1,1.02), loc='upper left')

plt.show()

In [ ]:
incident_file_clean_df.head()

In [ ]:
incident_file_clean_df.shape

HYPOTHESIS TESTING (ANJI)

In [ ]:
import scipy.stats as stats

In [ ]:
# Create Boxplot
response_df.boxplot(column="Response Time", by="Year", showfliers=False, figsize=(20,10), fontsize=20)
plt.ylim(0, 20)
plt.title("")
plt.suptitle("LVFD Response Times per Year", fontsize=25)
plt.xlabel("Year", fontsize=20)
plt.ylabel("Response Times (minutes)", fontsize=20)

plt.show()

In [ ]:
# Extract individual groups
group1 = response_df[response_df["Year"] == 2011]["Response Time"]
group2 = response_df[response_df["Year"] == 2012]["Response Time"]
group3 = response_df[response_df["Year"] == 2013]["Response Time"]
group4 = response_df[response_df["Year"] == 2014]["Response Time"]
group5 = response_df[response_df["Year"] == 2015]["Response Time"]
group6 = response_df[response_df["Year"] == 2016]["Response Time"]
group7 = response_df[response_df["Year"] == 2017]["Response Time"]
group8 = response_df[response_df["Year"] == 2018]["Response Time"]

In [ ]:
# Perform the ANOVA
statistics = stats.f_oneway(group1, group2, group3, group4, group5, group6, group7, group8)
print(statistics)

In [ ]:
# Bring-in fire station data
station_df = pd.read_csv(fire_station_data)
print(f"# of rows in the Station data file: {len(station_df)}")
station_df.head()

In [ ]:
#clean up data 
distinct_stations = incident_file_clean_df["Station"].unique()
clv_stations_df = station_df[station_df["CITY_CODE"] == "CLV"]
clv_stations_df.head()

In [ ]:

#Access maps with unique API key
gmaps.configure(api_key=g_key)


In [ ]:
#create a map of Firehouse locations
station_locations = clv_stations_df[['LAT','LONG']]

fig = gmaps.figure()
markers = gmaps.marker_layer(station_locations)


   
fig.add_layer(markers)
fig

In [ ]:
# create markers
info_box_template = """
<dl>
<dt>Facility Name</dt><dd>{FACILITY_NAME}</dd>
<dt>Latitude</dt><dd>{LAT}</dd>
<dt>Longitude</dt><dd>{LONG}</dd>
</dl>
"""
# Store the DataFrame Row
# update with your DataFrame name
location_info = [info_box_template.format(**row) for index, row in clv_stations_df.iterrows()]
locations = clv_stations_df[["LAT", "LONG"]]

In [ ]:
# Add marker layer ontop of map
markers = gmaps.marker_layer(locations,info_box_content=location_info)
fig.add_layer(markers)
fig